# Data De-duplication

Record linkage, also known as deduplication, is a crucial step in data cleaning tasks. It involves identifying and linking records that refer to the same entity across different datasets. This code implements record linkage using the recordlinkage library in Python. We first clean and pre-process the data, including address standardization and removal of irrelevant information. Then, we employ various blocking techniques to narrow down potential matches based on shared attributes like state or surname. Further, we define custom comparison functions to measure string similarity for specific attributes like names and addresses. Jaro-Winkler distance is used for names and addresses, while Levenshtein distance is chosen for attributes like postal codes and dates. Finally, the code filters candidate record pairs based on user-defined similarity thresholds, resulting in a refined list of potential matches for further review or classification.

This has several usecases. One sample use case:
An individual holds a property portfolio with purcahses spread over the last 30 years. During that time, the individual changed address, contact info, surname, and possibly made inconsistent entries for addresses (123 Kensington St. instead of 123 Kensington Street, etc.). There is even a mistake in date of birth where 10/06/1960 is written as 06/10/1960. Furthermore, some properties are in joint ownership with his wife and some with his son. If the tax office needs to combine all the assets for that person, a simple rule based matching might not work. Treating these records separately will lead to potential tax loss for the department. A model that can match such records and link them to a single person would help recover lost taxes.

In [1]:
import re
import string
import pandas as pd

from recordlinkage.datasets import load_febrl2
import recordlinkage

In [2]:
df = load_febrl2()
df.head(20)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-2778-org,sarah,bruhn,44,forbes street,wintersloe,kellerberrin,4510,vic,19300213,7535316
rec-712-dup-0,jacob,lanyon,5,milne cove,wellwod,beaconsfield upper,2602,vic,19080712,9497788
rec-1321-org,brinley,efthimiou,35,sturdee crescent,tremearne,scarborough,5211,qld,19940319,6814956
rec-3004-org,aleisha,hobson,54,oliver street,inglewood,toowoomba,3175,qld,19290427,5967384
rec-1384-org,ethan,gazzola,49,sheaffe street,bimby vale,port pirie,3088,sa,19631225,3832742
rec-3981-org,alicia,hope,100,mansfield place,sunset,byford,6061,sa,19421201,7934773
rec-916-org,benjamin,kolosche,78,keenan street,wingara,raymond terrace,3212,sa,19450918,5698873
rec-1684-org,petreece,lusted,78,baseby place,NaN,east branxton,6214,sa,19950620,8084076
rec-63-dup-0,olivia,white,55,duffy street,shopping village,mirrabooka,2260,vic,19000106,4996142


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-2778-org to rec-1143-org
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     4891 non-null   object
 1   surname        4936 non-null   object
 2   street_number  4777 non-null   object
 3   address_1      4891 non-null   object
 4   address_2      4431 non-null   object
 5   suburb         4950 non-null   object
 6   postcode       5000 non-null   object
 7   state          4952 non-null   object
 8   date_of_birth  4890 non-null   object
 9   soc_sec_id     5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


Postcode does not have any null values and could be considered as an exact match for

In [4]:
df.describe()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
count,4891,4936,4777,4891,4431,4950,5000,4952,4890,5000
unique,907,1816,397,2455,2517,1684,1412,19,3785,4089
top,emiily,white,1,forbes street,rosetta village,toowoomba,4701,nsw,19920917,1135598
freq,85,131,142,16,40,46,23,1604,7,6


In [5]:
# Change to lower case
df = df.astype(str).apply(lambda x: x.str.lower())
df.head(20)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-2778-org,sarah,bruhn,44,forbes street,wintersloe,kellerberrin,4510,vic,19300213,7535316
rec-712-dup-0,jacob,lanyon,5,milne cove,wellwod,beaconsfield upper,2602,vic,19080712,9497788
rec-1321-org,brinley,efthimiou,35,sturdee crescent,tremearne,scarborough,5211,qld,19940319,6814956
rec-3004-org,aleisha,hobson,54,oliver street,inglewood,toowoomba,3175,qld,19290427,5967384
rec-1384-org,ethan,gazzola,49,sheaffe street,bimby vale,port pirie,3088,sa,19631225,3832742
rec-3981-org,alicia,hope,100,mansfield place,sunset,byford,6061,sa,19421201,7934773
rec-916-org,benjamin,kolosche,78,keenan street,wingara,raymond terrace,3212,sa,19450918,5698873
rec-1684-org,petreece,lusted,78,baseby place,nan,east branxton,6214,sa,19950620,8084076
rec-63-dup-0,olivia,white,55,duffy street,shopping village,mirrabooka,2260,vic,19000106,4996142


### Remove stopwords in the context of the dataset

In this dataset, stopwords would likely appear in address fields only and can be removed from the dataframe.

In [6]:
address_stopwords = ['STREET', 'ST', 'PLACE', 'RD', 'ROAD']

In [7]:

##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TEXT


#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = text.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub("\\.(?=\\s|$)", "", text)
    #text = re.sub("[^a-z]", " ", text)
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text


#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(text):
    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text)) #remove last hashtags
    text = " ".join(word.strip() for word in re.split('#|_', text)) #remove hashtags symbol from words in the middle of the sentence
    return text

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

# Remove stop words (for address)
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in address_stopwords])


## Getting only the numerical value for postcode (can be used for mobile numbers too)
def clean_col_as_number(col_name):
    df[col_name] = df[col_name].str.strip()
    df[col_name] = df[col_name].str.findall('[0-9]+')
    df[col_name] = df[col_name].str.join("")
    df[col_name] = df[col_name].fillna("")

In [8]:
# Clean each address field
cleaned_address1 = []
cleaned_address2 = []

for address in df.address_1:
    cleaned_address1.append(remove_stopwords(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(address))))))

for address in df.address_2:
    cleaned_address2.append(remove_stopwords(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(address))))))


# Clean postcodes as it contains some characters such as + and -
clean_col_as_number('postcode')
clean_col_as_number('date_of_birth')


# Add cleaned addresses as new columns
df['clean_address_1'] = cleaned_address1
df['clean_address_2'] = cleaned_address2

df.head()

 Comparing a record with other records can beomce computationally expensive and time consuming. We will create indexes using different techniques in the recordlinkage library to reduce these combinations:

- **Full Index**: This creates an index containing all possible pairs of records from the DataFrame. While comprehensive, it's computationally expensive for large datasets.
- **Sorted Neighbourhood**: This indexes records based on a shared attribute (e.g., surname). It considers records with similar values within a specified window, reducing the number of comparisons.
- **Block Index**: This indexes records based on a categorical attribute (e.g., state). It only considers records from the same block (category), further narrowing down potential matches.

we then combine these indexes and removes duplicate pairs that might arise due to overlapping blocking criteria.


In [10]:
dup_indexer = recordlinkage.Index()
dup_indexer.full()
full_index_table = dup_indexer.index(df)

print(f"Table records: {len(df)} records, No of Pairs: {len(full_index_table)} pairs")

Table records: 5000 records, No of Pairs: 12497500 pairs


Firstly we will create a SortedNeighbourhood Indexer. This approach considers records with similar surnames within a window of 5, reducing the number of pairs to compare while focusing on potentially similar records.


In [11]:
from recordlinkage.index import SortedNeighbourhood

# Create a SortedNeighbourhood indexer
sorted_neighbor_indexer = SortedNeighbourhood(left_on="surname", right_on="surname", window = 5)

# Index the data using sorted neighborhoods based on surname
sorted_neighbor_index_table = sorted_neighbor_indexer.index(df)

print(f"Table records: {len(df)} records, No of Pairs: {len(sorted_neighbor_index_table)} pairs")

Table records: 5000 records, No of Pairs: 75034 pairs


Next, we create Block indexer on state. This approach considers records from the same state, further reducing the number of pairs to compare and focusing on geographically close records.

In [12]:
from recordlinkage.index import Block

block_indexer = Block(left_on='state', right_on='state')
block_index_table = block_indexer.index(df)

print(f"Table records: {len(df)} records, No of Pairs: {len(block_index_table)} pairs")

Table records: 5000 records, No of Pairs: 2768103 pairs


Merging both block and sorted neighbor

In [20]:
# Combine block and sorted neighborhood index pairs
all_index_pairs = block_index_table.append(sorted_neighbor_index_table)

# Remove duplicate pairs (might occur due to overlapping blocking criteria)
all_index_pairs = all_index_pairs.drop_duplicates(keep='first')

In [14]:
all_index_pairs

MultiIndex([('rec-712-dup-0',  'rec-2778-org'),
            ( 'rec-63-dup-0',  'rec-2778-org'),
            ( 'rec-63-dup-0', 'rec-712-dup-0'),
            (  'rec-112-org',  'rec-2778-org'),
            (  'rec-112-org', 'rec-712-dup-0'),
            (  'rec-112-org',  'rec-63-dup-0'),
            ( 'rec-2116-org',  'rec-2778-org'),
            ( 'rec-2116-org', 'rec-712-dup-0'),
            ( 'rec-2116-org',  'rec-63-dup-0'),
            ( 'rec-2116-org',   'rec-112-org'),
            ...
            ( 'rec-3587-org',  'rec-1541-org'),
            ( 'rec-3005-org',  'rec-3647-org'),
            (  'rec-322-org',   'rec-232-org'),
            (  'rec-322-org',  'rec-1304-org'),
            (  'rec-322-org',  'rec-1523-org'),
            (  'rec-322-org',  'rec-2049-org'),
            (  'rec-322-org',  'rec-2975-org'),
            (  'rec-322-org',   'rec-396-org'),
            ( 'rec-3812-org',  'rec-1485-org'),
            (  'rec-303-org',  'rec-2719-org')],
           names=['rec_

Create similarity measures for all the record pairs. We will use 2 methods for different types of fields
- Jaro-Winkler is generally better for names and addresses
- Levenshtein is more suitable for numbers and fixed-length strings

In [21]:
# Create a comparison object
dup_compare = recordlinkage.Compare()

# Define comparison functions for string attributes
dup_compare.string('given_name','given_name', method='jarowinkler', label = 'given_name_score')
dup_compare.string('surname','surname', method='jarowinkler', label = 'surname_score')
dup_compare.string('street_number','street_number', method='levenshtein', label = 'street_number_score')
dup_compare.string('clean_address_1','clean_address_1', method='jarowinkler', label = 'address_1_score')
dup_compare.string('clean_address_2','clean_address_2', method='jarowinkler', label = 'address_2_score')
dup_compare.string('suburb','suburb', method='jarowinkler', label = 'suburb_score')
dup_compare.string('postcode','postcode', method='levenshtein', label = 'postcode_score')
dup_compare.string('state','state', method='jarowinkler', label = 'state_score')
dup_compare.string('date_of_birth','date_of_birth', method='levenshtein', label = 'date_of_birth_score')
dup_compare.string('soc_sec_id','soc_sec_id', method='levenshtein', label = 'soc_sec_id_score')

# Compute comparison vectors for all index pairs using the defined comparison functions
comparison_vectors = dup_compare.compute(all_index_pairs, df)

Print the comparison of each of the record pairs

In [16]:
comparison_vectors.head(20)

given_name_score  surname_score  \
rec_id_1      rec_id_2                                         
rec-712-dup-0 rec-2778-org           0.466667       0.455556   
rec-63-dup-0  rec-2778-org           0.455556       0.000000   
              rec-712-dup-0          0.000000       0.000000   
rec-112-org   rec-2778-org           0.588889       0.633333   
              rec-712-dup-0          0.577778       0.000000   
              rec-63-dup-0           0.555556       0.000000   
rec-2116-org  rec-2778-org           0.447619       0.565079   
              rec-712-dup-0          0.447619       0.436508   
              rec-63-dup-0           0.539683       0.000000   
              rec-112-org            0.373016       0.428571   
rec-3232-org  rec-2778-org           0.577778       0.561905   
              rec-712-dup-0          0.455556       0.642857   
              rec-63-dup-0           0.000000       0.447619   
              rec-112-org            0.000000       0.464286   
              rec-2116-org           0.539683       0.523810   
rec-3123-org  rec-2778-org           0.550000       0.577778   
              rec-712-dup-0          0.550000       0.555556   
              rec-63-dup-0           0.527778       0.411111   
              rec-112-org            0.527778       0.000000   
              rec-2116-org           0.490079       0.436508   

                             street_number_score  address_1_score  \
rec_id_1      rec_id_2                                              
rec-712-dup-0 rec-2778-org                   0.0         0.510256   
rec-63-dup-0  rec-2778-org                   0.0         0.677350   
              rec-712-dup-0                  0.5         0.405556   
rec-112-org   rec-2778-org                   0.0         0.603276   
              rec-712-dup-0                  0.0         0.588889   
              rec-63-dup-0                   0.0         0.562169   
rec-2116-org  rec-2778-org                   0.5         0.632906   
              rec-712-dup-0                  0.0         0.555556   
              rec-63-dup-0                   0.0         0.540476   
              rec-112-org                    0.0         0.574074   
rec-3232-org  rec-2778-org                   0.0         0.514329   
              rec-712-dup-0                  0.0         0.545098   
              rec-63-dup-0                   0.0         0.522876   
              rec-112-org                    0.0         0.450980   
              rec-2116-org                   0.0         0.578525   
rec-3123-org  rec-2778-org                   0.0         0.740769   
              rec-712-dup-0                  0.0         0.555556   
              rec-63-dup-0                   0.0         0.650000   
              rec-112-org                    0.0         0.711111   
              rec-2116-org                   0.0         0.644444   

                             address_2_score  suburb_score  postcode_score  \
rec_id_1      rec_id_2                                                       
rec-712-dup-0 rec-2778-org          0.657143      0.514550            0.00   
rec-63-dup-0  rec-2778-org          0.547222      0.405556            0.25   
              rec-712-dup-0         0.401786      0.457407            0.50   
rec-112-org   rec-2778-org          0.665476      0.388889            0.25   
              rec-712-dup-0         0.535714      0.514550            0.25   
              rec-63-dup-0          0.502976      0.494444            0.25   
rec-2116-org  rec-2778-org          0.565079      0.462963            0.00   
              rec-712-dup-0         0.436508      0.472222            0.25   
              rec-63-dup-0          0.428571      0.403704            0.25   
              rec-112-org           0.619048      0.527778            0.50   
rec-3232-org  rec-2778-org          0.542294      0.477778            0.00   
              rec-712-dup-0         0.483487      0.514550            0.25   
         

In [17]:
comparison_vectors.describe()

,given_name_score,surname_score,street_number_score,address_1_score,address_2_score,suburb_score,postcode_score,state_score,date_of_birth_score,soc_sec_id_score
count,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06,2.825974e+06
mean,4.108766e-01,4.109149e-01,1.204130e-01,5.647552e-01,4.557543e-01,4.604112e-01,1.658027e-01,9.802272e-01,4.126615e-01,1.285831e-01
std,2.254969e-01,2.259160e-01,2.208189e-01,1.133888e-01,1.708870e-01,1.406506e-01,1.822654e-01,1.382954e-01,1.378851e-01,1.206603e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.166667e-01,4.111111e-01,0.000000e+00,4.990385e-01,4.222222e-01,4.179894e-01,0.000000e+00,1.000000e+00,3.750000e-01,0.000000e+00
50%,4.666667e-01,4.555556e-01,0.000000e+00,5.615385e-01,4.777778e-01,4.777778e-01,2.500000e-01,1.000000e+00,3.750000e-01,1.428571e-01
75%,5.500000e-01,5.396825e-01,2.500000e-01,6.305556e-01,5.357143e-01,5.352564e-01,2.500000e-01,1.000000e+00,5.000000e-01,1.428571e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Refine the candidate record linkages by applying stricter criteria on top of the initial blocking and indexing steps. You can adjust the similarity thresholds (0.75, 0.80 in this example) based on your data quality and desired level of stringency in identifying potential matches.

In [18]:
comparison_vectors[(comparison_vectors.soc_sec_id_score != 1) & (comparison_vectors.date_of_birth_score > 0.75) & (comparison_vectors.address_1_score > 0.75) & (comparison_vectors.given_name_score > 0.80)]

given_name_score  surname_score  \
rec_id_1       rec_id_2                                          
rec-2802-dup-1 rec-2802-dup-0          1.000000       1.000000   
               rec-2802-org            1.000000       1.000000   
rec-3980-org   rec-3980-dup-0          0.906667       1.000000   
rec-3379-dup-2 rec-3379-dup-3          1.000000       0.000000   
rec-3379-org   rec-3379-dup-3          1.000000       0.000000   
...                                         ...            ...   
rec-2756-dup-0 rec-2756-org            1.000000       1.000000   
rec-594-dup-2  rec-594-dup-3           0.960000       1.000000   
rec-594-dup-1  rec-594-dup-2           1.000000       1.000000   
rec-594-org    rec-594-dup-2           1.000000       1.000000   
rec-594-dup-2  rec-594-dup-0           1.000000       0.966667   

                               street_number_score  address_1_score  \
rec_id_1       rec_id_2                                               
rec-2802-dup-1 rec-2802-dup-0                  1.0         1.000000   
               rec-2802-org                    1.0         1.000000   
rec-3980-org   rec-3980-dup-0                  1.0         0.952564   
rec-3379-dup-2 rec-3379-dup-3                  0.0         1.000000   
rec-3379-org   rec-3379-dup-3                  1.0         1.000000   
...                                            ...              ...   
rec-2756-dup-0 rec-2756-org                    1.0         1.000000   
rec-594-dup-2  rec-594-dup-3                   1.0         0.925420   
rec-594-dup-1  rec-594-dup-2                   1.0         0.961667   
rec-594-org    rec-594-dup-2                   1.0         0.961667   
rec-594-dup-2  rec-594-dup-0                   1.0         0.961667   

                               address_2_score  suburb_score  postcode_score  \
rec_id_1       rec_id_2                                                        
rec-2802-dup-1 rec-2802-dup-0         0.984615      1.000000             1.0   
               rec-2802-org           1.000000      1.000000             1.0   
rec-3980-org   rec-3980-dup-0         0.983333      0.971429             0.5   
rec-3379-dup-2 rec-3379-dup-3         1.000000      1.000000             1.0   
rec-3379-org   rec-3379-dup-3         1.000000      1.000000             1.0   
...                                        ...           ...             ...   
rec-2756-dup-0 rec-2756-org           1.000000      1.000000             1.0   
rec-594-dup-2  rec-594-dup-3          0.971429      0.410256             0.5   
rec-594-dup-1  rec-594-dup-2          0.492063      0.410256             0.5   
rec-594-org    rec-594-dup-2          0.971429      0.410256             0.5   
rec-594-dup-2  rec-594-dup-0          0.971429      0.410256             0.5   

                               state_score  date_of_birth_score  \
rec_id_1       rec_id_2                                           
rec-2802-dup-1 rec-2802-dup-0     1.000000                  1.0   
               rec-2802-org       1.000000                  1.0   
rec-3980-org   rec-3980-dup-0     1.000000                  1.0   
rec-3379-dup-2 rec-3379-dup-3     1.000000                  1.0   
rec-3379-org   rec-3379-dup-3     1.000000                  1.0   
...                                    ...                  ...   
rec-2756-dup-0 rec-2756-org       0.555556                  1.0   
rec-594-dup-2  rec-594-dup-3      0.555556                  1.0   
rec-594-dup-1  rec-594-dup-2      0.555556                  1.0   
rec-594-org    rec-594-dup-2      0.555556                  1.0   
rec-594-dup-2  rec-594-dup-0      0.555556                  1.0   

                               soc_sec_id_score  
rec_id_1       rec_id_2                          
rec-2802-dup-1 rec-2802-dup-0          0.857143  
               rec-2802-org            0.857143  
rec-3980-org   rec-3980-dup-0          0.714286  
rec-3379-dup-2 rec-3379-dup-3          0.857143  
rec-3379-org   rec-3379-dup-3      

Test a sample record pair listed above to see the actual data in both of them

In [19]:
df.filter(items = ['rec-3949-dup-0','rec-3949-org'], axis=0)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,clean_address_1,clean_address_2
rec_id,,,,,,,,,,,,
rec-3949-dup-0,bayde,son,4290,warramboolfclose,st francks vlge,blackstone,2488,qld,19390328,1372776,warramboolfclose,st francks vlge
rec-3949-org,bayden,son,4290,warrambool close,st francis vlge,blackstone,2848,qld,19390328,9100309,warrambool close,st francis vlge
